In [1]:
# %load train_feature_extraction.py
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
import numpy as np
# TODO: Load traffic signs data.
# Load pickled data
import pickle
import sys
# import cPickle
import math
# to set the plt to show true gray scale images set this parameter 
# plt.rcParams['image.cmap'] = 'gray'
# TODO: fill this in based on where you saved the training and testing data
training_file = '/media/pemfir/Data/docker/lab 2 data/train2.p'
testing_file = '/media/pemfir/Data/docker/lab 2 data/test2.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)


# TODO: Split data into training and validation sets.
nb_classes = 43
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
# X_train = (X_train-128.0)/128.0
# X_test = (X_test-128.0)/128.0
X_train, X_validation, y_train, y_validation  = train_test_split(X_train, y_train , test_size=0.33, random_state=0)
from sklearn.utils import shuffle


# X_train = tf.image.resize_images(X_train, (227, 227))
# X_test = tf.image.resize_images(X_test, (227, 227))
def flatMaker(batch_x):
    return np.reshape( batch_x ,([-1,np.prod(X_train.get_shape().as_list()[1:])]))
# TODO: Define placeholders and resize operation.
# x = tf.placeholder(tf.float32,(None,np.prod(X_train.get_shape().as_list()[1:])))
# y = tf.placeholder(tf.int32, (None),name='labels')
# Xshape = X_train.shape
x = tf.placeholder(tf.float32,(None,32,32,3))
resized = tf.image.resize_images(x, (227, 227))
y = tf.placeholder(tf.int64, (None))
one_hot_y = tf.one_hot(y, 43)
# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

# TODO: Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
weight4 = tf.Variable(tf.truncated_normal(shape,stddev = 0.01))
bias4 = tf.Variable(tf.zeros(43))
logits = tf.nn.xw_plus_b(fc7, weight4, bias4)
# probs = tf.nn.softmax(logits)
# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.

loss_operation = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y))

optimizer = tf.train.AdamOptimizer()
# optimization - minimize cross entropy 

training_operation = optimizer.minimize(loss_operation, var_list=[weight4,bias4] )
# finding the class with largest logit and comparing it against the true lable one_hot_y

# correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
correct_prediction = tf.equal(tf.argmax(logits, 1), y)

# averages of 0s and 1s from the boolean operation above gives you the accuracy 
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# TODO: Train and evaluate the feature extraction model.

EPOCHS = 10

BATCH_SIZE = 128
init_op = tf.initialize_all_variables()
def evaluate(X_data, y_data):
    num_examples = X_data.shape[0]
    print(num_examples)
    total_accuracy, total_loss = 0, 0
    # --todo-- learn more about what sessions are and what the line below means. 
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
#         batch_x, batch_y = imageGenerator(batch_x, batch_y,2)
#         batch_x  = flatMaker(batch_x)
        # we want to report both the loss and accuracy so we call both operations here
        loss, accuracy =  sess.run([loss_operation, accuracy_operation], feed_dict={x: batch_x,
                                                                                    y: batch_y})
        # weighted average of accuracies by batch size gives us the total average accuracy 
        total_accuracy += (accuracy * batch_x.shape[0])
        # weighted average of losses , where the weights are size of the batches give us total loss 
        total_loss     += (loss * batch_x.shape[0])
    print(total_loss)
    print(total_accuracy)
    return total_loss / num_examples, total_accuracy / num_examples


with tf.Session() as sess:
    sess.run(init_op)
#     writer = tf.summary.FileWriter("logs/",sess.graph)
    num_examples = X_train.shape[0]   
    print("Training...")
    print()
    for i in range(EPOCHS):
#             X_trainScaled, y_train = shuffle(X_trainScaled, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            X_train, y_train = shuffle(X_train, y_train)
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
#                 batch_x, batch_y = imageGenerator(batch_x, batch_y,10)
#             batch_x  = flatMaker(batch_x)
#                 print('batch number {}'.format(offset))
            # calling training operation and storing output into variable loss 
            # regularization only appears in training 
            loss = sess.run([training_operation], feed_dict={x: batch_x, y: batch_y})
#             filter_summary = tf.image_summary(tf.Variable(initial_value=layerOneFilter))

#             filter_summary = writer.add_summary(layerOneFilter)
        # once an EPOCH is finished we evaluate the model performance with drop out prob = 1.0 on validation data 
        validation_loss, validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Loss     = {:.3f}".format(validation_loss))
        print("Validation Accuracy = {:.4f}".format(validation_accuracy))
        print()
#     filter_summary = tf.summary.image(tensor = layerOneFilter, name = 'amir')  
#     if 0 == 0:
        # Log a summary every 10 steps.
#     writer.add_summary(filterSummary,1)
    # once the final model is ready we run evaluate again with drop out prob 1.0 on test data     
#     test_loss, test_accuracy = evaluate(X_test, y_test)
#     print("Test Loss     = {:.3f}".format(test_loss))
#     print("Test Accuracy = {:.4f}".format(test_accuracy))
    

# try:
#     saver
# except NameError:
#     saver = tf.train.Saver()
# saver.save(sess, 'lenet')
# print("Model saved")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training...

12939
6867.7726112
11191.0000003
EPOCH 1 ...
Validation Loss     = 0.531
Validation Accuracy = 0.8649

12939
4640.24956816
11726.0000006
EPOCH 2 ...
Validation Loss     = 0.359
Validation Accuracy = 0.9063

12939
3599.25203499
11967.0000006
EPOCH 3 ...
Validation Loss     = 0.278
Validation Accuracy = 0.9249

12939
3107.62895116
12077.0000003
EPOCH 4 ...
Validation Loss     = 0.240
Validation Accuracy = 0.9334

12939
2585.59984082
12301.0000003
EPOCH 5 ...
Validation Loss     = 0.200
Validation Accuracy = 0.9507

12939
2287.11752889
12343.0
EPOCH 6 ...
Validation Loss     = 0.177
Validation Accuracy = 0.9539

12939
2073.66612257
12396.0
EPOCH 7 ...
Validation Loss     = 0.160
Validation Accuracy = 0.9580

12939
1956.09985802
12406.0000003
EPOCH 8 ...
Validation Loss     = 0.151
Validation Accuracy = 0.9588

12939
1796.08495849
12442.0000003
EPOCH 9 ...
Validation Loss     = 0.139
Validation Accuracy 

In [ ]:
num_examples

In [3]:
# %load train_feature_extraction.py
import pickle
import time

import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
import numpy as np
from sklearn.utils import shuffle
# TODO: Load traffic signs data.
# Load pickled data
import pickle
import sys
# import cPickle
import math
# to set the plt to show true gray scale images set this parameter 
# plt.rcParams['image.cmap'] = 'gray'
# TODO: fill this in based on where you saved the training and testing data
training_file = '/media/pemfir/Data/docker/lab 2 data/train2.p'
testing_file = '/media/pemfir/Data/docker/lab 2 data/test2.p'

with open(training_file, mode='rb') as f:
    data = pickle.load(f)


nb_classes = 43
epochs = 10
batch_size = 128

with open('./train.p', 'rb') as f:
    data = pickle.load(f)

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)

features = tf.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.placeholder(tf.int64, None)
resized = tf.image.resize_images(features, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.initialize_all_variables()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))


def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    print(X.shape[0])
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])
    print(total_loss)
    print(total_acc)
    return total_loss/X.shape[0], total_acc/X.shape[0]

with tf.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
12939
6990.32409495
11062.0000006
Epoch 1
Time: 38.993 seconds
Validation Loss = 0.540252267946
Validation Accuracy = 0.854934693608

12939
4517.45740536
11775.0000006
Epoch 2
Time: 38.754 seconds
Validation Loss = 0.349134972205
Validation Accuracy = 0.910039415766



KeyboardInterrupt: 

In [ ]:
12939
27716.9255967
5420.00000012
EPOCH 1 ...
Validation Loss     = 2.142
Validation Accuracy = 0.4189


12939
6990.32409495
11062.0000006
Epoch 1
Time: 38.993 seconds
Validation Loss = 0.540252267946
Validation Accuracy = 0.854934693608